In [1]:
from PIL import Image, ImageDraw, ImageFont
import pytesseract
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import numpy as np
import joblib as jb

In [2]:
def process_image(image_name):
	return pytesseract.pytesseract.image_to_string(image_name, lang="eng", config="-c tessedit_char_whitelist=0123456789.")

# Detect Clock Text

In [11]:
# c, s = [], []
# for i in range(20, 392):
#     im_1 = Image.open(f"/home/shubhamch/project/NetColab/Videos/belgium_traces/P-musher/fr_{str(i).zfill(4)}.png")
#     g = process_image(im_1)[:-2]
#     c.append(g)
#     s.append(len(g))
#     # print(generated_text)

In [6]:
# For a single video

root = "../Results/Videos/"
trace = "Live/"
# videos = ["Video2", "Video3", "Video4", "Video5"]
videos = ["live"]
# frames = [2900, 2200, 2600, 2200, 2900] # for bus trace
# frames = [2900, 1800, 2550, 2300, 2900] # for car trace
# frames = [2100, 2600, 2400, 2900] # for sp bus trace
# frames = [2150, 2600, 2300, 2900] # for sp car trace
frames = [2300] # live without camera
# schedulers = ["Pbflow", "PminRTT", "Pmusher", "Tbflow", "TminRTT", "Tmusher", "Tours"]
schedulers = ["Tours"]

for kk, video in enumerate(videos):
    for scheduler in schedulers:
        print(f"Doing for {video} and {scheduler} >>>>>>>> {frames[kk]}")
        # c, s = [], []
        # for i in range(100, frames[kk]):
        #     im_1 = Image.open(f"{root+trace+video}/{scheduler}/fr_{str(i).zfill(4)}.png")
        #     g = process_image(im_1)[:-2]
        #     g = g.replace("\n","")
        #     # c.append(g)
        #     # print(g, len(g))
        #     # if len(g) == 14:
        #         # s.append(g)
        #     s.append(g)
        # print(f"Done for {video} and {scheduler} >>>>>>>>")  
        # jb.dump(s, f"{root+trace+video}/detections/detections_pkl_{scheduler}_{video}.pkl")
            
        # Separates both the clock texts
        s = jb.load(f"{root+trace+video}/detections/detections_pkl_{scheduler}_{video}.pkl")
        s2 = []
        for g in s:
            # print(g)
            g = g.strip()   # removes extra spaces, if any
            if(len(g)==14):
                n=7
            if(len(g)==7):
                n=5
            if(len(g)==12):
                n=6
            out = [(g[i:i+n]) for i in range(0, len(g), n)]
            s2.append(out)
            
        # # Calculates the difference to get the E2E lag
        j, latency = 0, []
        for i in s2:
            try:
                if(float(i[0]) and float(i[1])):
                    current_latency = float(i[1]) - float(i[0])
                    if current_latency<30 and current_latency>0:
                        latency.append(current_latency)
                        # print(current_latency)
            except:
                print("Error")
        jb.dump(latency, f"{root+trace+video}/lag_pkl_{scheduler}_{video}.pkl")

Doing for live and Tours >>>>>>>> 2300


## For online class

In [7]:
# Generates detection pickle file for all

root = "../Results/Videos/"
# traces = ["bus/", "car/", "walk/"]
traces = ["walk/"]
# videos = ["Video2", "Video3", "Video4", "Video5"]
videos = ["Video5"]
# frames = [[2100, 2600, 2200, 2800], [1800, 2550, 2300, 2900], [2100, 2500, 2300, 2900]] 
frames = [[2400]] 
# frames = [2900, 2200, 2600, 2200, 2900] # for bus trace
# frames = [2900, 1800, 2550, 2300, 2900] # for car trace
# frames = [2100, 2600, 2400, 2900] # for sp bus trace
# frames = [2150, 2600, 2300, 2900] # for sp car trace
schedulers = ["PSP1", "PSP2", "Pbflow", "PminRTT", "Pmusher", "Tbflow", "TminRTT", "Tmusher", "Tours"]

for tt, trace in enumerate(traces):
    for kk, video in enumerate(videos):
        if trace == "walk" and video == "Video2":
            continue
        for scheduler in schedulers:
            # if scheduler != "PSP1" and scheduler != "PSP2":
            #     continue
            print(f"Doing for {trace} {video} and {scheduler} >>>>>>>> {frames[tt][kk]}")
            s = np.full(frames[tt][kk] - 100, None)
            for i in range(100, frames[tt][kk]):
                im_1 = Image.open(f"{root+trace+video}/{scheduler}/fr_{str(i).zfill(4)}.png")
                g = process_image(im_1)[:-2]
                g = g.replace("\n","")
                # c.append(g)
                # print(g, len(g))
                # if len(g) == 14:
                    # s.append(g)
                s[i - 100] = g
            print(f"Done for {trace} {video} and {scheduler} >>>>>>>>")    
            jb.dump(s, f"{root+trace+video}/detections/detections_pkl_{scheduler}_{video}.pkl")

Doing for walk/ Video5 and Tours >>>>>>>> 2400
Done for walk/ Video5 and Tours >>>>>>>>


In [11]:
# Calculates latency and saves as pickle file

root = "../Results/Videos/"
traces = ["bus/", "car/", "walk/"]
videos = ["Video2", "Video3", "Video4", "Video5"]
frames = [[2100, 2600, 2200, 2800], [1800, 2550, 2300, 2900], [2100, 2500, 2300, 2900]] 
schedulers = ["PSP1", "PSP2", "Pbflow", "PminRTT", "Pmusher", "Tbflow", "TminRTT", "Tmusher", "Tours"]

for tt, trace in enumerate(traces):
    for kk, video in enumerate(videos):
        for scheduler in schedulers:
            # if video in ["Video2", "Video3", "Video4", "Video5"] and scheduler == "Pbflow" and trace == "walk/":
            #     continue 
            print(f"Doing for {trace} {video} and {scheduler} >>>>>>>> {frames[tt][kk]}")           
            # Separates both the clock texts
            s2 = []
            s = jb.load(f"{root+trace+video}/detections/detections_pkl_{scheduler}_{video}.pkl")
            for gg in range(len(s)):
                if s[gg] is not None:
                    s[gg] = s[gg].strip()   # removes extra spaces if any
                    if(len(s[gg])==14):
                        n=7
                    if(len(s[gg])==7):
                        n=5
                        # print("Hit 7")
                    if(len(s[gg])==12):
                        n=6
                        # print("Hit 12")
                    out = [(s[gg][i:i+n]) for i in range(0, len(s[gg]), n)]
                    s2.append(out)
        
            # # Calculates the difference to get the E2E lag
            j, latency = 0, []
            for i in s2:
                try:
                    if(float(i[0]) and float(i[1])):
                        current_latency = float(i[1]) - float(i[0])
                        if current_latency<30 and current_latency>0:
                            latency.append(current_latency)
                            # print(current_latency)
                except:
                    print("Error")
            jb.dump(latency, f"{root+trace+video}/lag_new_pkl_{scheduler}_{video}.pkl") 

Doing for walk/ Video5 and Tours >>>>>>>> 2400


## For new applications

In [ ]:
# Generates detection pickle file for all

root = "Videos/"
traces = ["bus/"]
videos = ["Badminton", "Tennis", "USNews"]
schedulers = ["minrtt", "compact"]

frames = 1080
# frames = [2900, 2200, 2600, 2200, 2900] # for bus trace
# frames = [2900, 1800, 2550, 2300, 2900] # for car trace
# frames = [2100, 2600, 2400, 2900] # for sp bus trace
# frames = [2150, 2600, 2300, 2900] # for sp car trace

for tt, trace in enumerate(traces):
    for kk, video in enumerate(videos):
        for scheduler in schedulers:
            print(f"Doing for {trace} {video} and {scheduler} >>>>>>>> {frames}")
            s = np.full(frames - 100, None)
            for i in range(100, frames):
                im_1 = Image.open(f"{root+trace+video}/{scheduler}/fr_{str(i).zfill(4)}.png")
                g = process_image(im_1)[:-2]
                g = g.replace("\n","")
                # c.append(g)
                # print(g, len(g))
                # if len(g) == 14:
                    # s.append(g)
                s[i - 100] = g
            print(f"Done for {trace} {video} and {scheduler} >>>>>>>>")
            Path(f"{root+trace+video}/detections/").mkdir(exist_ok=True, parents=True)
            jb.dump(s, f"{root+trace+video}/detections/detections_pkl_{scheduler}_{video}.pkl")

Doing for bus/ Badminton and minrtt >>>>>>>> 1080
Done for bus/ Badminton and minrtt >>>>>>>>
Doing for bus/ Badminton and compact >>>>>>>> 1080
Done for bus/ Badminton and compact >>>>>>>>
Doing for bus/ Tennis and minrtt >>>>>>>> 1080


In [ ]:
# Calculates latency and saves as pickle file

root = "../Results/Videos/"
traces = ["bus/", "car/", "walk/"]
videos = ["Video2", "Video3", "Video4", "Video5"]
frames = [[2100, 2600, 2200, 2800], [1800, 2550, 2300, 2900], [2100, 2500, 2300, 2900]] 
schedulers = ["PSP1", "PSP2", "Pbflow", "PminRTT", "Pmusher", "Tbflow", "TminRTT", "Tmusher", "Tours"]

for tt, trace in enumerate(traces):
    for kk, video in enumerate(videos):
        for scheduler in schedulers:
            # if video in ["Video2", "Video3", "Video4", "Video5"] and scheduler == "Pbflow" and trace == "walk/":
            #     continue 
            print(f"Doing for {trace} {video} and {scheduler} >>>>>>>> {frames[tt][kk]}")           
            # Separates both the clock texts
            s2 = []
            s = jb.load(f"{root+trace+video}/detections/detections_pkl_{scheduler}_{video}.pkl")
            for gg in range(len(s)):
                if s[gg] is not None:
                    s[gg] = s[gg].strip()   # removes extra spaces if any
                    if(len(s[gg])==14):
                        n=7
                    if(len(s[gg])==7):
                        n=5
                        # print("Hit 7")
                    if(len(s[gg])==12):
                        n=6
                        # print("Hit 12")
                    out = [(s[gg][i:i+n]) for i in range(0, len(s[gg]), n)]
                    s2.append(out)
        
            # # Calculates the difference to get the E2E lag
            j, latency = 0, []
            for i in s2:
                try:
                    if(float(i[0]) and float(i[1])):
                        current_latency = float(i[1]) - float(i[0])
                        if current_latency<30 and current_latency>0:
                            latency.append(current_latency)
                            # print(current_latency)
                except:
                    print("Error")
            jb.dump(latency, f"{root+trace+video}/lag_new_pkl_{scheduler}_{video}.pkl") 

In [40]:
latency
# s2

[4.760000000000019]